In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.89 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 40.7/112.6 GB disk)


In [ ]:
import os
import cv2
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define input and output dataset paths
train_dataset_path = "/content/drive/MyDrive/DSGP/Data/train/images"
processed_train_dataset_path = "/content/drive/MyDrive/DSGP/Model9/Data/train/images"

valid_dataset_path = "/content/drive/MyDrive/DSGP/Data/valid/images"
processed_valid_dataset_path = "/content/drive/MyDrive/DSGP/Model9/Data/valid/images"

test_dataset_path = "/content/drive/MyDrive/DSGP/Data/test/images"
processed_test_dataset_path = "/content/drive/MyDrive/DSGP/Model9/Data/test/images"

# Ensure the output directory exists
os.makedirs(processed_train_dataset_path, exist_ok=True)

def apply_fourier_transform(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale

    # Perform FFT and shift zero frequency to center
    fft_image = np.fft.fft2(gray_image)
    fft_shifted = np.fft.fftshift(fft_image)

    # Design a High-Pass Filter
    rows, cols = gray_image.shape
    crow, ccol = rows // 2, cols // 2
    radius = 30
    mask = np.ones((rows, cols), dtype=np.float32)
    y, x = np.ogrid[:rows, :cols]
    mask_area = (x - ccol)**2 + (y - crow)**2 <= radius**2
    mask[mask_area] = 0

    # Apply High-Pass Filter
    filtered_fft = fft_shifted * mask

    # Perform Inverse FFT
    ifft_shifted = np.fft.ifftshift(filtered_fft)
    filtered_image = np.fft.ifft2(ifft_shifted)
    filtered_image = np.abs(filtered_image)  # Get magnitude

    # Normalize to 0-255 range
    filtered_image = cv2.normalize(filtered_image, None, 0, 255, cv2.NORM_MINMAX)

    # Convert to 3-channel (YOLO requires 3-channel images)
    processed_image = cv2.merge([filtered_image] * 3)

    return processed_image.astype(np.uint8)

Loop through train images
for img_name in os.listdir(train_dataset_path):
    img_path = os.path.join(train_dataset_path, img_name)
    image = cv2.imread(img_path)

    if image is None:
        continue  # Skip unreadable images

    processed_image = apply_fourier_transform(image)  # Apply Fourier Transform

    # Save processed image
    save_path = os.path.join(processed_train_dataset_path, img_name)
    cv2.imwrite(save_path, processed_image)


Loop through valid images
for img_name in os.listdir(valid_dataset_path):
    img_path = os.path.join(valid_dataset_path, img_name)
    image = cv2.imread(img_path)

    if image is None:
        continue  # Skip unreadable images

    processed_image = apply_fourier_transform(image)  # Apply Fourier Transform

    # Save processed image
    save_path = os.path.join(processed_valid_dataset_path, img_name)
    cv2.imwrite(save_path, processed_image)

# Loop through test images
for img_name in os.listdir(test_dataset_path):
    img_path = os.path.join(test_dataset_path, img_name)
    image = cv2.imread(img_path)

    if image is None:
        continue  # Skip unreadable images

    processed_image = apply_fourier_transform(image)  # Apply Fourier Transform

    # Save processed image
    save_path = os.path.join(processed_test_dataset_path, img_name)
    cv2.imwrite(save_path, processed_image)

print("All images have been processed and saved.")

All images have been processed and saved.


In [ ]:
from ultralytics import YOLO

In [ ]:
%cd /content/drive/MyDrive/DSGP/Model9

/content/drive/MyDrive/DSGP/Model9


In [ ]:
!yolo task=detect mode=train model=yolov5n.pt data=data.yaml epochs=100 imgsz=640 plots=True batch=16 lr0=0.005

PRO TIP 💡 Replace 'model=yolov5n.pt' with new 'model=yolov5nu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.

100% 5.31M/5.31M [00:00<00:00, 83.8MB/s]
Ultralytics 8.3.89 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov5n.pt, data=data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, 

In [ ]:
!yolo task=detect mode=val model=runs/detect/train5/weights/best.pt data=data.yaml

Ultralytics 8.3.89 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv5n summary (fused): 84 layers, 2,503,139 parameters, 0 gradients, 7.1 GFLOPs
val: Scanning /content/drive/MyDrive/DSGP/Model9/Data/valid/labels.cache... 150 images, 50 backgrounds, 0 corrupt: 100% 150/150 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 10/10 [00:03<00:00,  3.24it/s]
                   all        150        481      0.327      0.141      0.115     0.0362
Speed: 1.7ms preprocess, 7.3ms inference, 0.0ms loss, 3.1ms postprocess per image
Results saved to runs/detect/val2
💡 Learn more at https://docs.ultralytics.com/modes/val


In [ ]:
!yolo task=detect mode=predict model=runs/detect/train5/weights/best.pt conf=0.25 source=/content/drive/MyDrive/DSGP/Model9/Data/test/images

Ultralytics 8.3.89 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv5n summary (fused): 84 layers, 2,503,139 parameters, 0 gradients, 7.1 GFLOPs

image 1/200 /content/drive/MyDrive/DSGP/Model9/Data/test/images/007a7f83-0001596.png: 384x640 (no detections), 37.8ms
image 2/200 /content/drive/MyDrive/DSGP/Model9/Data/test/images/01ca576d-D02_20210628090856_0000225.png: 384x640 (no detections), 7.8ms
image 3/200 /content/drive/MyDrive/DSGP/Model9/Data/test/images/04f031bd-D02_20210628090856_0000229.png: 384x640 (no detections), 30.7ms
image 4/200 /content/drive/MyDrive/DSGP/Model9/Data/test/images/07a1d4d0-D01_20210502070930_0033775.png: 384x640 (no detections), 8.4ms
image 5/200 /content/drive/MyDrive/DSGP/Model9/Data/test/images/081fc519-D02_20210628090856_0000226.png: 384x640 (no detections), 7.8ms
image 6/200 /content/drive/MyDrive/DSGP/Model9/Data/test/images/085594e0-D02_20210628090856_0000069.png: 384x640 (no detections), 6.8ms
image 7/200 /content/drive/MyDrive/D